# <font color="purple"><b>Grid Search CV Algorithm for Fully Connected Networks</b></font>

Using the grid search CV algorithm, the hyperparameters of this model is sought.
<li><b> Learning Rate:</b> 0.1, 0.01</li>
<li><b> Regularization Penalty:</b>0.01, 0.001, 0.0001</li>
<li><b> Batch Size:</b> 64, 128</li>
<li><b> Epochs:</b> 50, 100, 150</li>

## <font color="blue">Import Libraries</font>

In [ ]:
import gzip
import pickle
import numpy as np
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
import tensorflow
import json
import cv2
import io
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
try:
    to_unicode = unicode
except NameError:
    to_unicode = str
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
!pip install hickle
import hickle as hkl

In [ ]:
data = hkl.load("data.hkl")
X_train, X_test, Y_train, y_test = data['xtrain'], data['xtest'], data['ytrain'], data['ytest']
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [ ]:
from tensorflow.data import Dataset
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D,Input,MaxPooling2D, Dense, Dropout, MaxPool1D, Flatten, AveragePooling1D, BatchNormalization
from tensorflow.keras import Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential


In [ ]:
def model_1(weight_decay):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(32, 32, 1), kernel_regularizer=l2(weight_decay)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(4, activation='softmax', kernel_regularizer=l2(weight_decay)))
  return model


def model_2(weight_decay):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(32, 32, 1), kernel_regularizer=l2(weight_decay)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(4, activation='softmax', kernel_regularizer=l2(weight_decay)))
  return model


def model_3(weight_decay):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(32, 32, 1),kernel_regularizer=l2(weight_decay)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Conv2D(256, kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(4, activation='softmax',kernel_regularizer=l2(weight_decay)))
  return model


<font color="blue"> The algorithm below is that ... </font>

In [ ]:
from sklearn.metrics import confusion_matrix
def encoded_label(y_predict):
  y_list = [] 
  for y_hat in y_predict:
    y_hat = np.argmax(y_hat)
    y_list.append(to_categorical(y_hat))
  return y_list


def KFold_GridSearchCV(input_dim, X, Y, X_test, y_test, combinations, filename="log.csv", acc_loss_json="hist.json"):
    """Summary: Grid Search CV for 3 Folds Cross Validation
    """
    res_df = pd.DataFrame(columns=['momentum','learning rate','batch size',
                                      'loss1', 'acc1','loss2', 'acc2','loss3', 'acc3', 'widing factor',
                                      'prec1', 'prec2', 'prec3', 'recall1', 'recall2', 'recall3'])
    generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)
    hist_dict_global = {}

    for i, combination in enumerate(combinations):
        kf = KFold(n_splits=3, random_state=42, shuffle=False)
        metrics_dict = {}
  
        for j, (train_index, test_index) in enumerate(kf.split(X)):
            X_train, X_val = X[train_index], X[test_index]
            y_train, y_val = Y[train_index], Y[test_index]
            model = model_1(combination[2])
            opt = tensorflow.keras.optimizers.SGD(learning_rate=combination[0])
            model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
            hist = model.fit(generator.flow(X_train, y_train, batch_size=combination[1]), steps_per_epoch=len(X_train) // combination[1], epochs=combination[3],
                                      validation_data=(X_val, y_val),
                                      validation_steps=len(X_val) // combination[1],)
            loss, acc = model.evaluate(X_test, y_test)
            #yhat_classes = encoded_label( model.predict(X_test))
            predict =  model.predict(X_test)
            yhat_classes = np.argmax(predict, axis=1)
            print(yhat_classes)
            print(y_test)
            cm = confusion_matrix(np.argmax(y_test, axis=1), yhat_classes)
            recall = np.diag(cm) / np.sum(cm, axis = 1)
            precision = np.diag(cm) / np.sum(cm, axis = 0)

            recall_avg = np.mean(recall)
            precision_avg = np.mean(precision)
            metrics_dict[j+1] = {"loss": loss, "acc": acc, "epoch_stopped": combination[3], "precision_avg":precision_avg,
                                 "avg_recall":recall_avg}
            graph_loss_acc = {"id": i, "com":j+1, "val_acc":hist.history["val_accuracy"], "train_acc":hist.history["accuracy"],
                                    "val_loss":hist.history["val_loss"], "train_loss":hist.history["loss"], "epoch_stopped": combination[3], 'learning rate': combination[0],
                                    'batch size': combination[1], 'reg_penalty': combination[2]}

              # Write JSON file
            with io.open(acc_loss_json, 'a+', encoding='utf8') as outfile:
                str_ = json.dumps(graph_loss_acc)
                outfile.write(to_unicode(str_))
        row = {'momentum': combination[4],'learning rate': combination[0],
               'batch size': combination[1],
               'reg_penalty': combination[2],
               'epoch_stopped': metrics_dict[1]["epoch_stopped"],
               'widing factor' : 1,
               'loss1': metrics_dict[1]["loss"],
               'acc1': metrics_dict[1]["acc"],
               'loss2': metrics_dict[2]["loss"],
               'acc2': metrics_dict[2]["acc"],
               'loss3': metrics_dict[3]["loss"],
               'acc3': metrics_dict[3]["acc"],
               'prec1':metrics_dict[1]["precision_avg"],
               'prec2':metrics_dict[2]["precision_avg"],
               'prec3':metrics_dict[3]["precision_avg"],
               'recall1':metrics_dict[1]["avg_recall"],
               'recall2':metrics_dict[2]["avg_recall"],
               'recall3':metrics_dict[3]["avg_recall"]}
        res_df = res_df.append(row , ignore_index=True)
        res_df.to_csv(filename, sep=";")

In [ ]:

if __name__ == "__main__":
    learning_rate = [0.1,0.01]
    batch_size = [64,128]
    reg_penalty = [0,0.01, 0.001, 0.0001]
    epochs = [50,100,150]
    momentum = [0.9]
    in_dim = (32,32,1)
    grid_result = "grid_model_1.csv"
    acc_loss_json = "history.json"
    # create list of all different parameter combinations
    param_grid = dict(learning_rate = learning_rate, batch_size = batch_size, 
                      reg_penalty = reg_penalty, epochs = epochs, momentum=momentum)
    combinations = list(product(*param_grid.values()))
    KFold_GridSearchCV(in_dim,X_train,Y_train,X_test, y_test, combinations, grid_result, acc_loss_json)

In [ ]:
data  = pd.read_csv("grid_model_1.csv", sep=";")

In [ ]:
data.head(5)

,momentum,learning rate,batch size,loss1,acc1,loss2,acc2,loss3,acc3,widing factor,prec1,prec2,prec3,recall1,recall2,recall3,epoch_stopped,reg_penalty
0,0.9,0.1,64.0,1.078664,0.541012,1.087765,0.560209,1.140540,0.495637,1.0,0.545625,0.570303,0.540447,0.549296,0.561227,0.506039,50.0,0.00
1,0.9,0.1,64.0,1.090459,0.544503,1.052103,0.568935,0.991915,0.586387,1.0,0.567745,0.581882,0.587054,0.549632,0.571559,0.584749,100.0,0.00
2,0.9,0.1,64.0,1.203270,0.549738,1.024475,0.607330,1.099551,0.542757,1.0,0.597314,0.629707,0.583353,0.558664,0.612029,0.532126,150.0,0.00
3,0.9,0.1,64.0,1.331875,0.450262,1.357389,0.425829,1.136434,0.542757,1.0,0.520104,0.556441,0.541896,0.467984,0.441258,0.549730,50.0,0.01
4,0.9,0.1,64.0,1.286225,0.490401,1.218342,0.539267,1.207052,0.537522,1.0,0.552838,0.569478,0.586226,0.494587,0.541650,0.547532,100.0,0.01


In [ ]:
data["loss_mean"] = (data["loss1"]+data["loss2"]+data["loss3"])/3
data["acc_mean"] = (data["acc1"]+data["acc2"]+data["acc3"])/3

In [ ]:
data['epoch'] = data['epoch_stopped']
data['weight_decay'] = data['reg_penalty']

In [ ]:
data['recall_mean'] = (data['recall1']+data['recall2']+data['recall3'])/3
data['prec_mean'] = (data['prec1']+data['prec2']+data['prec3'])/3

In [ ]:
column_list = ["momentum", "learning rate", "epoch","batch size","weight_decay","loss_mean", "acc_mean","recall_mean", "prec_mean"]

In [ ]:
data.sort_values(axis=0, by="loss_mean", ascending=True)[column_list].head(3)

,momentum,learning rate,epoch,batch size,weight_decay,loss_mean,acc_mean,recall_mean,prec_mean
24,0.9,0.01,50.0,64.0,0.0000,1.012071,0.612565,0.614360,0.615845
34,0.9,0.01,100.0,64.0,0.0001,1.026681,0.614311,0.612435,0.619908
8,0.9,0.10,150.0,64.0,0.0010,1.027309,0.603258,0.601262,0.606730


In [ ]:
data["loss_na"] = data.loc[:,["loss1","loss2", "loss3"]].isnull().sum(1)

In [ ]:
data.head(3)

,momentum,learning rate,batch size,loss1,acc1,loss2,acc2,loss3,acc3,widing factor,prec1,prec2,prec3,recall1,recall2,recall3,epoch_stopped,reg_penalty,loss_mean,acc_mean,epoch,weight_decay,loss_na
0,0.9,0.1,64.0,1.078664,0.541012,1.087765,0.560209,1.140540,0.495637,1.0,0.545625,0.570303,0.540447,0.549296,0.561227,0.506039,50.0,0.0,1.102323,0.532286,50.0,0.0,0
1,0.9,0.1,64.0,1.090459,0.544503,1.052103,0.568935,0.991915,0.586387,1.0,0.567745,0.581882,0.587054,0.549632,0.571559,0.584749,100.0,0.0,1.044826,0.566609,100.0,0.0,0
2,0.9,0.1,64.0,1.203270,0.549738,1.024475,0.607330,1.099551,0.542757,1.0,0.597314,0.629707,0.583353,0.558664,0.612029,0.532126,150.0,0.0,1.109098,0.566608,150.0,0.0,0


In [ ]:
generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

In [ ]:
kf = KFold(n_splits=3, random_state=42, shuffle=False)
result = []
for j, (train_index, test_index) in enumerate(kf.split(X_train)):
  x_train, x_val = X_train[train_index], X_train[test_index]
  y_train, y_val = Y_train[train_index], Y_train[test_index]
  model = model_2(0)
  opt = tensorflow.keras.optimizers.SGD(learning_rate=0.01)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
  hist = model.fit(generator.flow(x_train, y_train, batch_size=64), steps_per_epoch=len(x_train) //64 , epochs=50,
                   validation_data=(x_val, y_val),
                  validation_steps=len(x_val) //64 ,)

  test = model.evaluate(X_test, y_test)
  result.append(test)

In [ ]:
mean_acc = (result[0][1]+result[1][1]+result[2][1])/3;mean_acc

0.6305991808573405

In [ ]:
mean_loss = (result[0][0]+result[1][0]+result[2][0])/3;mean_loss

0.9891296029090881

In [ ]:
kf = KFold(n_splits=3, random_state=42, shuffle=False)
result_2 = []
for j, (train_index, test_index) in enumerate(kf.split(X_train)):
  x_train, x_val = X_train[train_index], X_train[test_index]
  y_train, y_val = Y_train[train_index], Y_train[test_index]
  model = model_3(0.0001)
  opt = tensorflow.keras.optimizers.SGD(learning_rate=0.01)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
  hist = model.fit(generator.flow(x_train, y_train, batch_size=64), steps_per_epoch=len(x_train) //64 , epochs=100,
                   validation_data=(x_val, y_val),
                  validation_steps=len(x_val) //64 ,)

  test = model.evaluate(X_test, y_test)
  result_2.append(test)

In [ ]:
mean_acc = (result_2[0][1]+result_2[1][1]+result_2[2][1])/3;mean_acc

0.7108784119288126

In [ ]:
mean_loss = (result_2[0][0]+result_2[1][0]+result_2[2][0])/3;mean_loss

0.9208946625391642

In [ ]:
mean_acc = (result_2[0][1]+result_2[1][1]+result_2[2][1])/3;mean_acc

0.6783013343811035

In [ ]:
mean_loss = (result_2[0][0]+result_2[1][0]+result_2[2][0])/3;mean_loss

0.8747362097104391

In [ ]:
model = model_1(0)
opt = tensorflow.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
model.summary()

Model: "sequential_162"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_342 (Conv2D)          (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_343 (Conv2D)          (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_183 (MaxPoolin (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_183 (Bat (None, 14, 14, 64)        256       
_________________________________________________________________
flatten_162 (Flatten)        (None, 12544)             0         
_________________________________________________________________
dense_162 (Dense)            (None, 4)                 50180     
Total params: 69,252
Trainable params: 69,124
Non-trainable params: 128
______________________________________________

In [ ]:
model = model_2(0)
opt = tensorflow.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
model.summary()

Model: "sequential_154"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_320 (Conv2D)          (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_321 (Conv2D)          (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_166 (MaxPoolin (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_166 (Bat (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_322 (Conv2D)          (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_167 (MaxPoolin (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_167 (Bat (None, 6, 6, 128)      

In [ ]:
model = model_3(0)
opt = tensorflow.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics= ['accuracy'])
model.summary()

Model: "sequential_155"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_323 (Conv2D)          (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_324 (Conv2D)          (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_168 (MaxPoolin (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_168 (Bat (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_325 (Conv2D)          (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_169 (MaxPoolin (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_169 (Bat (None, 6, 6, 128)      